In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from scipy import stats

# Load data
data = pd.read_csv("test.csv")  # assumes cleaned CSV with 561 features + label

# Separate features and target
X = data.drop("Activity", axis=1)
y = data["Activity"]

# Handle missing values


In [ ]:
imputer = SimpleImputer(strategy="mean")
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

In [ ]:
z_scores = np.abs(stats.zscore(X))
X = X[(z_scores < 3).all(axis=1)]
y = y[X.index]  # align labels

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
pca = PCA()
X_pca = pca.fit_transform(X_scaled)
explained_variance = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
n_components = np.argmax(explained_variance >= 0.95) + 1
print(f"Number of components explaining 95% variance: {n_components}")

Number of components explaining 95% variance: 89


In [ ]:
pca = PCA(n_components=n_components)
X_reduced = pca.fit_transform(X_scaled)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
mlp = MLPClassifier(max_iter=1000, random_state=42)
param_dist = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50), (50, 25)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [1e-4, 1e-3, 1e-2],
    'learning_rate': ['constant', 'adaptive']
}

In [ ]:
random_search = RandomizedSearchCV(
    mlp, param_distributions=param_dist,
    n_iter=20, cv=5, scoring='accuracy', random_state=42, n_jobs=-1
)
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=MLPClassifier(max_iter=1000, random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'activation': ['relu', 'tanh'],
                                        'alpha': [0.0001, 0.001, 0.01],
                                        'hidden_layer_sizes': [(50,), (100,),
                                                               (100, 50),
                                                               (50, 25)],
                                        'learning_rate': ['constant',
                                                          'adaptive'],
                                        'solver': ['adam', 'sgd']},
                   random_state=42, scoring='accuracy')

In [ ]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nBest Parameters:")
print(random_search.best_params_)
print("\nAccuracy on test set:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Best Parameters:
{'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (50,), 'alpha': 0.01, 'activation': 'tanh'}

Accuracy on test set: 0.9663461538461539

Classification Report:
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00        27
           SITTING       0.93      0.95      0.94        59
          STANDING       0.95      0.93      0.94        56
           WALKING       1.00      1.00      1.00        25
WALKING_DOWNSTAIRS       1.00      1.00      1.00        11
  WALKING_UPSTAIRS       1.00      1.00      1.00        30

          accuracy                           0.97       208
         macro avg       0.98      0.98      0.98       208
      weighted avg       0.97      0.97      0.97       208



In [ ]:
from sklearn.metrics import confusion_matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))



Confusion Matrix:
[[27  0  0  0  0  0]
 [ 0 56  3  0  0  0]
 [ 0  4 52  0  0  0]
 [ 0  0  0 25  0  0]
 [ 0  0  0  0 11  0]
 [ 0  0  0  0  0 30]]
